# Importing packages

In [5]:
from hyperopt import tpe
from hyperopt import STATUS_OK, Trials, hp, fmin
import pandas as pd 
import numpy as np 
import mlflow
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, confusion_matrix 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

In [6]:
df = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv', header=None)
df.head()

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


# Data Preprocessing

In [7]:
X = df.iloc[:,:-1]
Y = df.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=0, stratify=Y)
mx = MinMaxScaler()

X_train = mx.fit_transform(X_train)
X_test = mx.transform(X_test)


# Define Objective Function

In [8]:
N_FOLDS = 4 
MAX_EVALS = 10
def objective(params, n_folds = N_FOLDS):
    mlflow.sklearn.autolog()
    with mlflow.start_run(nested=True):
        clf = LogisticRegression(**params, random_state=0)
        scores = cross_val_score(clf, X_train, y_train, cv=N_FOLDS, scoring='f1_macro')
        
        best_score = max(scores)
        
        loss = 1 - best_score
        
        return {'loss': loss, 'params':params, 'status': STATUS_OK}

# Define parameter space

In [9]:
space ={
    'warm_start': hp.choice('warm_start', [True, False]),
    'fit_intercept': hp.choice('fit_intercept',[True, False]),
    'tol': hp.uniform('tol', 0.00001, 0.0001),
    'C': hp.uniform('C', 0.05, 3),
    'solver': hp.choice('solver', ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']),
    'max_iter': hp.choice('max_iter', range(5,1000))
}

# Create experiment

In [10]:
mlflow.set_experiment('Hyperopt_Optimization')

INFO: 'Hyperopt_Optimization' does not exist. Creating a new experiment


# Define Optimization trials

In [11]:
# Algorithm
tpe_algorithm = tpe.suggest

bayes_trials = Trials()
with mlflow.start_run(run_name='hyper_opt_logistic') as run: 
    best = fmin(fn=objective, space=space, algo=tpe_algorithm, max_evals=MAX_EVALS, trials=bayes_trials)
    best = {k:float(v) for k,v in best.items()}
    mlflow.log_dict(best, "best_params.json")

100%|██████████| 10/10 [01:53<00:00, 11.39s/trial, best loss: 0.23139843581832542]
